# Stedin Distribution Transformer

In [1]:
using gmsh

using LinearAlgebra, SparseArrays
using WriteVTK

using BenchmarkTools

# Geometry Definition
See exercise on Gmsh.

In [2]:
# Enclosure dimensions (enclosure is not centered, the core is)
hencl1 = 65.5e-2;   # Height above the x-axis
hencl2 = -53.5e-2;  # Height below the y-axis
wencl  = 104e-2;    # Width

# Core dimensions
wcore = 84e-2;
hcore = 100e-2;

# Core gap dimensions (left and right are identical)
wgap = 17e-2;
hgap = 76e-2;
mgap = 17e-2;

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Load Geometry

In [3]:
gmsh.initialize()

In [4]:
# mesh_data struct
#  saves the nodes and elements of a gmsh geometry, for easy passing around in functions
struct mesh_data
    nnodes  # number of nodes
    xnode   # array of x coordinates
    ynode   # array of y coordinates
    
    nelements   # number of elements
    element_connectivity  # array of connectivity for each element
    e_group     # array containing the physical group number of each element
    elements    # <new addition> more conveniently structured connectivity array
end

# Loads nodes, elements, and element physical groups from gmsh and stores them in a mesh_data struct
function get_mesh_data()
    #..2/11 Get and sort the mesh nodes
    #..Observe that although the mesh is two-dimensional,
    #..the z-coordinate that is equal to zero is stored as well.
    #..Observe that the coordinates are stored contiguously for computational
    #..efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    
    #..4/12 Get the mesh elements
    #..observe that we get all the two-dimensional triangular elements from the mesh
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
    nelements = length(element_ids[1])

    #..5/12 Create groups of elements for the subdomains
    #..for loop that creates a vector describing which physical group an element belongs to
    ngroup1 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 1)
    ngroup2 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 2)
    ngroup3 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 3)
    ngroup4 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 4)
    ngroup5 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 5)
    ngroup6 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 6)
    ngroup7 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 7)
    ngroup8 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 8)
    ngroup9 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 9)
    ngroup10 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 10)
    ngroup11 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 11)
    ngroup12 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 12)
    ngroup13 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 13)
    ngroup14 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 14)
    ngroup15 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 15)
    ngroup16 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 16)
    e_group = zeros(1,nelements)
    
    for element_id in 1:nelements
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
        G1  = sum(node1_id.== ngroup1[1])+sum(node2_id.== ngroup1[1])+sum(node3_id.== ngroup1[1]) # Oil
        G2  = sum(node1_id.== ngroup2[1])+sum(node2_id.== ngroup2[1])+sum(node3_id.== ngroup2[1]) # Core
        G3  = sum(node1_id.== ngroup3[1])+sum(node2_id.== ngroup3[1])+sum(node3_id.== ngroup3[1]) # HV winding phase 1 left
        G4  = sum(node1_id.== ngroup4[1])+sum(node2_id.== ngroup4[1])+sum(node3_id.== ngroup4[1]) # HV winding phase 1 right
        G5  = sum(node1_id.== ngroup5[1])+sum(node2_id.== ngroup5[1])+sum(node3_id.== ngroup5[1]) # HV winding phase 2 left
        G6  = sum(node1_id.== ngroup6[1])+sum(node2_id.== ngroup6[1])+sum(node3_id.== ngroup6[1]) # HV winding phase 2 right
        G7  = sum(node1_id.== ngroup7[1])+sum(node2_id.== ngroup7[1])+sum(node3_id.== ngroup7[1]) # HV winding phase 3 left
        G8  = sum(node1_id.== ngroup8[1])+sum(node2_id.== ngroup8[1])+sum(node3_id.== ngroup8[1]) # HV winding phase 3 right
        G9  = sum(node1_id.== ngroup9[1])+sum(node2_id.== ngroup9[1])+sum(node3_id.== ngroup9[1]) # LV winding phase 1 left
        G10 = sum(node1_id.== ngroup10[1])+sum(node2_id.== ngroup10[1])+sum(node3_id.== ngroup10[1]) # LV winding phase 1 right
        G11 = sum(node1_id.== ngroup11[1])+sum(node2_id.== ngroup11[1])+sum(node3_id.== ngroup11[1]) # LV winding phase 2 left
        G12 = sum(node1_id.== ngroup12[1])+sum(node2_id.== ngroup12[1])+sum(node3_id.== ngroup12[1]) # LV winding phase 2 right
        G13 = sum(node1_id.== ngroup13[1])+sum(node2_id.== ngroup13[1])+sum(node3_id.== ngroup13[1]) # LV winding phase 3 left
        G14 = sum(node1_id.== ngroup14[1])+sum(node2_id.== ngroup14[1])+sum(node3_id.== ngroup14[1]) # LV winding phase 3 right
        
        if G1 == 3
            e_group[element_id] = 1;
        elseif G2 == 3
            e_group[element_id] = 2;
        elseif G3 == 3
            e_group[element_id] = 3;
        elseif G4 == 3
            e_group[element_id] = 4;
        elseif G5 == 3
            e_group[element_id] = 5;
        elseif G6 == 3
            e_group[element_id] = 6;
        elseif G7 == 3
            e_group[element_id] = 7;
        elseif G8 == 3
            e_group[element_id] = 8;
        elseif G9 == 3
            e_group[element_id] = 9;
        elseif G10 == 3
            e_group[element_id] = 10;
        elseif G11 == 3
            e_group[element_id] = 11;
        elseif G12 == 3
            e_group[element_id] = 12;
        elseif G13 == 3
            e_group[element_id] = 13;
        elseif G14 == 3
            e_group[element_id] = 14;
        end
    end
    
    elids = 1:nelements;
    els = [zeros(Int, 3) for i in elids];
    econn = element_connectivity;
    for eid in elids
        els[eid] = [econn[1][3*(eid-1)+1], econn[1][3*(eid-1)+2], econn[1][3*(eid-1)+3]]
    end
    
    return mesh_data(nnodes, xnode, ynode, nelements, element_connectivity, e_group, els)
end

get_mesh_data (generic function with 1 method)

In [5]:
function fem2d(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    A = spzeros(Complex{Float64}, mshdata.nnodes, mshdata.nnodes)
    f = zeros(Complex{Float64}, mshdata.nnodes, 1)

    element_connectivity = mshdata.element_connectivity;
    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    #..8/12 Perform a loop over the elements
    for element_id in 1:mshdata.nelements
        #....retrieve global numbering of the local nodes of the current element
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]

        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        #....compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        #....compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

        #....compute local matrix contribution Aloc of the current element
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Emat[3,:] .= 0;
        Bloc = area_id*reluctivityperelement[element_id]*(transpose(Emat)*Emat);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        #....add local contribution to f and A
        I      = element_connectivity[1][3 * (element_id - 1) .+ (1:3)];
        f[I]   += floc;
        A[I,I] += (Bloc + Cloc);
    end

    #..9/12 Handle the boundary conditions
    bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
    f[bnd_node_ids] .= 0;
    
    #..10/12 Compute the numerical solution
    u = A \ f
    
    return u;
end

fem2d (generic function with 1 method)

In [6]:
function process(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    Bx = zeros(Complex{Float64}, mshdata.nelements);
    By = zeros(Complex{Float64}, mshdata.nelements);
    
    Jel = zeros(mshdata.nelements);
    
    for element_id in 1:mshdata.nelements
        # Get nodes consituting the element
        node1_id = mshdata.element_connectivity[1][3*(element_id-1)+1]
        node2_id = mshdata.element_connectivity[1][3*(element_id-1)+2]
        node3_id = mshdata.element_connectivity[1][3*(element_id-1)+3]
        
        # Get x and y coordinates of the three nodes
        xnode = mshdata.xnode; ynode = mshdata.ynode;
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Calculate shape function parameters
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
    
        # Calculate Bx and By from the solution coefficients and the shape function parameters
        c = u[[node1_id, node2_id, node3_id]];
        Bx[element_id] = sum(c .* Emat[2,:]);
        By[element_id] = -sum(c .* Emat[1,:]);
        
        # Calculate eddy current loss
        sigma = conductivityperelement[element_id];
        Jel[element_id] = norm(sourceperelement[element_id] + omega * sigma * 1/3 * sum(c));
    end
    
    # H is related to B through the reluctivity
    Hx = reluctivityperelement' .* Bx;
    Hy = reluctivityperelement' .* By;
    
    # Energy is 0.5 * dot(B, H)
    Wm = 0.5 * (Bx .* Hx .+ By .* Hy);
    
    return (Bx,By), (Hx, Hy), Wm, Jel;
end

process (generic function with 1 method)

# Linear FEM without Eddy Currents

In [7]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

Info    : Reading 'geo/transformer_stedin.msh'...
Info    : 168 entities
Info    : 8004 nodes
Info    : 16006 elements
Info    : Done reading 'geo/transformer_stedin.msh'


In [8]:
Ip = 0*17.54;       # Primary peak phase current
Is = 777.62;      # Secondary peak phase current
Np = 266;
Ns = 6;
omega = 2*pi*50;

Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

sourcefunction(group_id) = Jp * (-1 * (group_id==3) + 1 * (group_id==4)) + Jp * exp(1im * 2pi/3) * (-1 * (group_id==5) + 1 * (group_id==6)) + Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + Js * (1 * (group_id==9) - 1 * (group_id==10)) + Js * exp(1im * 2pi/3) * (1 * (group_id==11) - 1 * (group_id==12)) + Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14))
sourceperelement = map(sourcefunction, mshdata.e_group);

mu0 = 4e-7 * pi;
mur = 1000;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group);

conductivityfunction(group_id) = 0;
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [9]:
u = fem2d(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega);
B, H, Wm, Jel = process(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

In [10]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, mshdata.element_connectivity[1][3*(i-1).+(1:3)]) for i = 1:mshdata.nelements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer1.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;

# Save the file
outfiles = vtk_save(vtkfile);

![Magnetic Field: Linear, No Eddy Currents](images/transformer1.png)

# Linear FEM with Eddy Currents

In [11]:
Ip = 0*17.54;       # Primary peak phase current
Is = 777.62;      # Secondary peak phase current
Np = 266;
Ns = 6;
omega = 2*pi*50;

Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

sourcefunction(group_id) = Jp * (-1 * (group_id==3) + 1 * (group_id==4)) + Jp * exp(1im * 2pi/3) * (-1 * (group_id==5) + 1 * (group_id==6)) + Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + Js * (1 * (group_id==9) - 1 * (group_id==10)) + Js * exp(1im * 2pi/3) * (1 * (group_id==11) - 1 * (group_id==12)) + Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14))
sourceperelement = map(sourcefunction, mshdata.e_group);

mu0 = 4e-7 * pi;
mur = 1000;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group);

sigma_core = 1000;
conductivityfunction(group_id) = sigma_core * (group_id == 2);
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [12]:
u = fem2d(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega);
B, H, Wm, Jel = process(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

In [13]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, mshdata.element_connectivity[1][3*(i-1).+(1:3)]) for i = 1:mshdata.nelements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer2.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;

# Save the file
outfiles = vtk_save(vtkfile);

![Magnetic Field: Linear, with Eddy Currents](images/transformer2.png)

# Non-Linear FEM

In [14]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

Info    : Reading 'geo/transformer_stedin.msh'...
Info    : 168 entities
Info    : 8004 nodes
Info    : 16006 elements
Info    : Done reading 'geo/transformer_stedin.msh'


In [15]:
Ip = 0*17.54;     # Primary peak phase current
Is = 777.62;      # Secondary peak phase current
Np = 266;
Ns = 6;
omega = 2*pi*50;

Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

sourcefunction(group_id) = Jp * (-1 * (group_id==3) + 1 * (group_id==4)) + Jp * exp(1im * 2pi/3) * (-1 * (group_id==5) + 1 * (group_id==6)) + Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + Js * (1 * (group_id==9) - 1 * (group_id==10)) + Js * exp(1im * 2pi/3) * (1 * (group_id==11) - 1 * (group_id==12)) + Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14))
sourceperelement = map(sourcefunction, mshdata.e_group);

# Permeability function
bh_a = 2.12e-4; 
bh_b = 7.358;
bh_c = 1.18e7;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));

fmu(group_id, B) = mu0 + (fmur_core(B) - 1) * mu0 * (group_id == 2);
reluctivityfunction(group_id, B) = 1 / fmu(group_id, B);

sigma_core = 1000;
conductivityfunction(group_id) = sigma_core * (group_id == 2);
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [16]:
# Pre-calculate the matrix contributions that are constant in the non-linear problem
function precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega)
    elids = 1:mshdata.nelements;
    
    Bmat = [zeros(9) for i in elids];
    Cmat = [zeros(Complex{Float64}, 9) for i in elids];
    Emat = [zeros(3,3) for i in elids];
    
    f = zeros(Complex{Float64}, mshdata.nnodes)
    
    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    #..8/12 Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        #....retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];

        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        #....compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        #....compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

        #....compute local matrix contribution Aloc of the current element
        Eloc = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Eloc[3,:] .= 0;
        Bloc = area_id*(transpose(Eloc)*Eloc);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        #....add local contribution to f and A
        f[nodes] += floc;
        
        Bmat[element_id] = Bloc[:];
        Cmat[element_id] = Cloc[:];
        Emat[element_id] = Eloc;
    end
    
    return Bmat, Cmat, Emat, f;
end

precalc_contribs (generic function with 1 method)

In [17]:
function assemble_matrices(u, I, J, Bmat, Cmat, Emat, N)
    c = map(el -> u[el], mshdata.elements)
    Bx = map((c, E) -> sum(c .* E[2, :]), c, Emat)
    By = map((c, E) -> -sum(c .* E[1, :]), c, Emat)
    Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
    reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);
    
    # Generate matrix contributions
    Vb = reduce(vcat, map((B, nu) -> B * nu, Bmat, reluctivityperelement));
    Vc = reduce(vcat, Cmat);
    V  = Vb + Vc;
    
    return sparse(I, J, V, N, N);
end

function apply_boundaries(A, bnd_node_ids, e)
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    #f[bnd_node_ids] .= 0;
    
    return A#, f;
end

function res!(R, u)
    A = assemble_matrices(u, I, J, Bmat, Cmat, Emat, N);
    A = apply_boundaries(A, bnd_node_ids, e);
    R[:] = A * u - f;
end

res! (generic function with 1 method)

In [18]:
Bmat, Cmat, Emat, f = precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega);

# Generate index vectors
I_ = reduce(vcat, view.(mshdata.elements, Ref([1, 2, 3, 1, 2, 3, 1, 2, 3])))
J_ = reduce(vcat, view.(mshdata.elements, Ref([1, 1, 1, 2, 2, 2, 3, 3, 3])))
N = mshdata.nnodes;

# Generate vectors for boundary condition 
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
e = Diagonal(ones(size(bnd_node_ids)));

In [19]:
function nonlinsolve(mshdata, I, J, Bmat, Cmat, Emat, N, bnd_node_ids, e, alpha)
    A = assemble_matrices(zeros(mshdata.nnodes), I, J, Bmat, Cmat, Emat, N);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    f[bnd_node_ids] .= 0;

    u = A \ f;

    du = 1e6;
    Niter = 1;

    tol = 1e-6;
    Nmax = 100;
    
    uhist = zeros(N);
    
    while (du > tol) && (Niter < Nmax)
        uprev = u;
        uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

        A = assemble_matrices(uhist, I, J, Bmat, Cmat, Emat, N);
        A = apply_boundaries(A, bnd_node_ids, e);
        
        u = A \ f;

        du = norm(u - uprev);
        Niter += 1;
        print("#$Niter: $du\n")
    end
    
    return u;
end

nonlinsolve (generic function with 1 method)

In [20]:
u = nonlinsolve(mshdata, I_, J_, Bmat, Cmat, Emat, N, bnd_node_ids, e, 0.9);

#2: 13.798004071815393
#3: 9.976374290484824
#4: 2.987069976414043
#5: 1.3754317678027481
#6: 1.3959104040678358
#7: 1.3769701199022863
#8: 1.3399762550019099
#9: 0.6473953408106017
#10: 0.3366114855356161
#11: 0.2378144721497089
#12: 0.16822994314044448
#13: 0.15301121247499685
#14: 0.16030862108219276
#15: 0.1533836302526732
#16: 0.12444396081230567
#17: 0.09464400287694247
#18: 0.07225446042290558
#19: 0.05554587060149519
#20: 0.04289383717616702
#21: 0.03348347783046953
#22: 0.02656382187232089
#23: 0.02143496943864279
#24: 0.017525238700377834
#25: 0.01444005245513014
#26: 0.011934496168188351
#27: 0.009864838224027268
#28: 0.008151534848602907
#29: 0.0067584830897322945
#30: 0.0056800472772379085
#31: 0.00492328249899321
#32: 0.004476802167567807
#33: 0.004280809934576528
#34: 0.004232917605845891
#35: 0.004228014831462115
#36: 0.004190813836367799
#37: 0.0040830824226049875
#38: 0.003896112237213802
#39: 0.0036398319899406902
#40: 0.00333338578697021
#41: 0.002998394348224261
#4

In [21]:
c = map(el -> u[el], mshdata.elements)
Bx = map((c, E) -> sum(c .* E[2, :]), c, Emat)
By = map((c, E) -> -sum(c .* E[1, :]), c, Emat)
Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);

In [22]:
B, H, Wm, Jel = process(mshdata, u, sourceperelement, reluctivityperelement', conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

In [23]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer3.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;
vtkfile["mu_r", VTKCellData()]  = 1 ./ (mu0 * reluctivityperelement);

# Save the file
outfiles = vtk_save(vtkfile);

![Magnetic Field: Non-linear, with Eddy Currents](images/transformer3.png)